In [7]:
#Importing the packages needed
import cv2
import numpy as np
from ultralytics import YOLO
import datetime
import logging
import csv
import os
from collections import defaultdict
from sklearn.cluster import DBSCAN

In [8]:
class CrowdDetector:
    def __init__(self, proximity_threshold=100, min_crowd_size=3, 
                 consecutive_frames=10, confidence_threshold=0.5):
        print("Initializing Crowd Detector...")
        
        # Initialize YOLO model
        try:
            self.model = YOLO('yolov8n.pt')
        except Exception as e:
            print(f"Error loading YOLO model: {e}")
            raise
        
        self.proximity_threshold = proximity_threshold
        self.min_crowd_size = min_crowd_size
        self.consecutive_frames = consecutive_frames
        self.confidence_threshold = confidence_threshold
        
        # Initialize tracking variables
        self.crowd_history = []
        self.persistent_crowds = defaultdict(int)
        
        # Setup output directory
        self.output_dir = 'crowd_detection_results'
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Setup CSV logging with timestamp in filename
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.csv_file = os.path.join(self.output_dir, f'crowd_detection_{timestamp}.csv')
        
        # Initialize CSV file with headers
        with open(self.csv_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Frame_Number', 'Person_Count_in_Crowd', 'Timestamp'])
        print(f"CSV file initialized at: {self.csv_file}")
        
        # Setup logging
        log_file = os.path.join(self.output_dir, f'detection_log_{timestamp}.txt')
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(message)s'
        )
        print(f"Log file initialized at: {log_file}")
    
    def log_crowd_detection(self, frame_number, crowd_size, crowd_center):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        try:
            with open(self.csv_file, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    frame_number,
                    crowd_size,
                    timestamp
                ])
            logging.info(f"Frame {frame_number}: Detected crowd of {crowd_size} people at location {crowd_center}")
            return True
        except Exception as e:
            print(f"Error logging crowd detection: {e}")
            return False
    
    def analyze_frame(self, frame, frame_number):
        # Run detection
        try:
            results = self.model(frame)[0]
            boxes = []
            
            # Filter for person detections
            for detection in results.boxes.data.tolist():
                x1, y1, x2, y2, conf, class_id = detection
                if int(class_id) == 0 and conf > self.confidence_threshold:
                    boxes.append([x1, y1, x2, y2])
            
            if boxes:
                print(f"Frame {frame_number}: Detected {len(boxes)} people")
            
            # Get person centers
            centers = np.array([[
                (box[0] + box[2]) / 2,
                (box[1] + box[3]) / 2
            ] for box in boxes])
            
            #using DBSCAN for identifying the crowds
            if len(centers) >= self.min_crowd_size:
                clustering = DBSCAN(
                    eps=self.proximity_threshold,
                    min_samples=self.min_crowd_size,
                    metric='euclidean'
                ).fit(centers)
                
                # Group points by cluster
                crowds = defaultdict(list)
                for idx, label in enumerate(clustering.labels_):
                    if label >= 0:
                        crowds[label].append(idx)
                
                # Update crowd history and check persistence
                current_crowds = [group for group in crowds.values() 
                                if len(group) >= self.min_crowd_size]
                
                # Store current frame's crowds
                self.crowd_history.append((frame_number, current_crowds))
                
                # Remove old frames from history
                while len(self.crowd_history) > self.consecutive_frames:
                    self.crowd_history.pop(0)
                
                # Check for persistent crowds
                if len(self.crowd_history) == self.consecutive_frames:
                    for crowd_indices in current_crowds:
                        crowd_centers = centers[crowd_indices]
                        crowd_center = np.mean(crowd_centers, axis=0)
                        
                        # Log the crowd detection
                        self.log_crowd_detection(
                            frame_number,
                            len(crowd_indices),
                            crowd_center
                        )
                        
                        # Visualize the crowd
                        crowd_color = (255, 0, 0)  # Red for crowd
                        
                        # Draw connections between crowd members
                        for i in range(len(crowd_centers)):
                            for j in range(i + 1, len(crowd_centers)):
                                pt1 = tuple(map(int, crowd_centers[i]))
                                pt2 = tuple(map(int, crowd_centers[j]))
                                cv2.line(frame, pt1, pt2, crowd_color, 2)
                        
                        # Draw crowd label
                        label_pos = tuple(map(int, crowd_center))
                        cv2.putText(
                            frame,
                            f"Crowd: {len(crowd_indices)} people",
                            label_pos,
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            crowd_color,
                            2
                        )
            
            # Draw individual detections
            for box in boxes:
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Add frame info
            cv2.putText(
                frame,
                f"Frame: {frame_number} | People: {len(boxes)}", 
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2
            )
            
            return frame
            
        except Exception as e:
            print(f"Error analyzing frame {frame_number}: {e}")
            return frame
    
    def process_video(self, video_path, output_path=None):
        print(f"\nProcessing video: {video_path}")
        
        try:
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                raise ValueError("Error opening video file")
            
            # Get properties of the video
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            
            print(f"Video properties:")
            print(f"- Resolution: {frame_width}x{frame_height}")
            print(f"- FPS: {fps}")
            print(f"- Total frames: {total_frames}")
            
            # Initialize video writer if output path is provided
            if output_path:
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                out = cv2.VideoWriter(
                    output_path,
                    fourcc,
                    fps,
                    (frame_width, frame_height)
                )
                print(f"Output video will be saved to: {output_path}")
            
            frame_number = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                frame_number += 1
                if frame_number % 30 == 0:  # Progress update every 30 frames
                    progress = (frame_number / total_frames) * 100
                    print(f"Processing progress: {progress:.1f}%")
                
                # Process frame
                processed_frame = self.analyze_frame(frame, frame_number)
                
                # Save output frame
                if output_path:
                    out.write(processed_frame)
                
                # Display frame
                cv2.imshow('Crowd Detection', processed_frame)
                
                # Break loop on 'q' press
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    print("\nProcessing interrupted by user")
                    break
            
            # Cleanup
            cap.release()
            if output_path:
                out.release()
            cv2.destroyAllWindows()
            
            print("\nProcessing completed!")
            print(f"Results saved to: {self.csv_file}")
            
        except Exception as e:
            print(f"Error processing video: {e}")
            raise

In [9]:
if __name__ == "__main__":
    try:
        # Create detector
        detector = CrowdDetector(
            proximity_threshold=100,
            min_crowd_size=3,
            consecutive_frames=10,
            confidence_threshold=0.5
        )
        
        # Process video
        input_video = 'dataset_video (1).mp4'  # Replace with your video path
        output_video = 'output_video.mp4'  # Replace with desired output path
        
        if not os.path.exists(input_video):
            raise FileNotFoundError(f"Input video not found: {input_video}")
        
        detector.process_video(input_video, output_video)
        
    except Exception as e:
        print(f"Error running crowd detection: {e}")

Initializing Crowd Detector...
CSV file initialized at: crowd_detection_results\crowd_detection_20241117_121324.csv
Log file initialized at: crowd_detection_results\detection_log_20241117_121324.txt

Processing video: dataset_video (1).mp4
Video properties:
- Resolution: 1920x1080
- FPS: 25
- Total frames: 341
Output video will be saved to: output_video.mp4

0: 384x640 37 persons, 2 birds, 75.0ms
Speed: 4.0ms preprocess, 75.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 1: Detected 22 people

0: 384x640 38 persons, 3 birds, 101.7ms
Speed: 4.5ms preprocess, 101.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Frame 2: Detected 21 people

0: 384x640 35 persons, 3 birds, 83.5ms
Speed: 3.0ms preprocess, 83.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 3: Detected 20 people

0: 384x640 37 persons, 3 birds, 79.5ms
Speed: 8.0ms preprocess, 79.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Frame 

Frame 48: Detected 17 people

0: 384x640 36 persons, 1 bird, 1 dog, 69.0ms
Speed: 4.0ms preprocess, 69.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 49: Detected 20 people

0: 384x640 36 persons, 1 bird, 1 dog, 81.5ms
Speed: 3.0ms preprocess, 81.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 50: Detected 20 people

0: 384x640 40 persons, 1 bird, 1 dog, 80.5ms
Speed: 3.0ms preprocess, 80.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 51: Detected 20 people

0: 384x640 41 persons, 1 bird, 1 dog, 75.5ms
Speed: 3.0ms preprocess, 75.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 52: Detected 19 people

0: 384x640 37 persons, 1 bird, 2 dogs, 84.5ms
Speed: 3.0ms preprocess, 84.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 53: Detected 21 people

0: 384x640 38 persons, 1 bird, 1 dog, 77.5ms
Speed: 3.0ms preprocess, 77.5ms inference, 1.0ms postprocess per

0: 384x640 37 persons, 78.5ms
Speed: 3.0ms preprocess, 78.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 99: Detected 22 people

0: 384x640 37 persons, 77.0ms
Speed: 5.0ms preprocess, 77.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 100: Detected 23 people

0: 384x640 35 persons, 76.5ms
Speed: 7.0ms preprocess, 76.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 101: Detected 23 people

0: 384x640 38 persons, 1 backpack, 66.5ms
Speed: 3.0ms preprocess, 66.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 102: Detected 23 people

0: 384x640 37 persons, 1 dog, 66.2ms
Speed: 3.0ms preprocess, 66.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 103: Detected 23 people

0: 384x640 38 persons, 1 bird, 73.6ms
Speed: 2.0ms preprocess, 73.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 104: Detected 22 people

0: 384x640 36 persons, 73.

Frame 148: Detected 23 people

0: 384x640 30 persons, 146.1ms
Speed: 4.0ms preprocess, 146.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 149: Detected 23 people
Processing progress: 44.0%

0: 384x640 30 persons, 120.9ms
Speed: 5.5ms preprocess, 120.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 150: Detected 25 people

0: 384x640 30 persons, 94.5ms
Speed: 3.0ms preprocess, 94.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 151: Detected 23 people

0: 384x640 30 persons, 1 bird, 101.5ms
Speed: 5.0ms preprocess, 101.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 152: Detected 23 people

0: 384x640 28 persons, 1 bird, 87.5ms
Speed: 4.0ms preprocess, 87.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 153: Detected 19 people

0: 384x640 29 persons, 85.5ms
Speed: 3.0ms preprocess, 85.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Fra

Frame 199: Detected 22 people

0: 384x640 32 persons, 1 handbag, 64.6ms
Speed: 3.0ms preprocess, 64.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 200: Detected 22 people

0: 384x640 34 persons, 1 dog, 72.6ms
Speed: 4.0ms preprocess, 72.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 201: Detected 21 people

0: 384x640 32 persons, 69.5ms
Speed: 2.0ms preprocess, 69.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 202: Detected 21 people

0: 384x640 32 persons, 1 motorcycle, 1 handbag, 65.0ms
Speed: 3.0ms preprocess, 65.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 203: Detected 22 people

0: 384x640 32 persons, 1 handbag, 75.6ms
Speed: 3.0ms preprocess, 75.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 204: Detected 21 people

0: 384x640 31 persons, 1 motorcycle, 1 handbag, 79.6ms
Speed: 4.0ms preprocess, 79.6ms inference, 2.0ms postprocess per image

Frame 246: Detected 15 people

0: 384x640 25 persons, 1 motorcycle, 1 bird, 66.0ms
Speed: 3.0ms preprocess, 66.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 247: Detected 15 people

0: 384x640 25 persons, 1 motorcycle, 1 handbag, 74.6ms
Speed: 2.0ms preprocess, 74.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 248: Detected 16 people

0: 384x640 24 persons, 2 birds, 1 handbag, 73.5ms
Speed: 4.0ms preprocess, 73.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 249: Detected 13 people

0: 384x640 25 persons, 2 birds, 1 dog, 71.5ms
Speed: 3.0ms preprocess, 71.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 250: Detected 15 people

0: 384x640 25 persons, 2 birds, 1 dog, 1 handbag, 91.5ms
Speed: 5.0ms preprocess, 91.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 251: Detected 14 people

0: 384x640 25 persons, 1 handbag, 107.5ms
Speed: 8.0ms preprocess, 10

Frame 294: Detected 16 people

0: 384x640 29 persons, 72.0ms
Speed: 4.0ms preprocess, 72.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 295: Detected 19 people

0: 384x640 30 persons, 1 backpack, 69.5ms
Speed: 2.0ms preprocess, 69.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 296: Detected 18 people

0: 384x640 32 persons, 1 backpack, 76.5ms
Speed: 4.0ms preprocess, 76.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 297: Detected 19 people

0: 384x640 30 persons, 1 bird, 1 backpack, 61.6ms
Speed: 3.0ms preprocess, 61.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 298: Detected 18 people

0: 384x640 30 persons, 1 bird, 1 dog, 1 backpack, 64.0ms
Speed: 2.0ms preprocess, 64.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Frame 299: Detected 16 people
Processing progress: 88.0%

0: 384x640 31 persons, 1 bird, 63.0ms
Speed: 2.0ms preprocess, 63.0ms inference, 